In [1]:
import helper_tools.parser as parser
import importlib
import pandas as pd

importlib.reload(parser)

relation_df, entity_df, docs = parser.redfm_parser("train")
predicate_set_df = relation_df[["predicate", "predicate_uri"]].drop_duplicates()

Fetching 22 files:   0%|          | 0/22 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:00<00:00, 12303.62it/s]


In [6]:
from dotenv import load_dotenv
from gen_ai_hub.proxy.langchain.openai import ChatOpenAI
from gen_ai_hub.proxy.core.proxy_clients import get_proxy_client

load_dotenv()

proxy_client = get_proxy_client('gen-ai-hub')
model = ChatOpenAI(proxy_model_name='meta--llama3-70b-instruct', proxy_client=proxy_client)

# Supervisor Agent

In [7]:
import re

text = "Olaf Scholz is the chancellor of Germany."
system_prompt = f"""
You are the Supervisor coordinating a conversation between two agents: entity_extraction_agent and relation_extraction_agent. Your goal is to extract information from the given text and convert it into triples for insertion into a knowledge graph.

You can use exactly one of the following actions per response:
1. Call an agent using: <goto>agent_name</goto> (where agent_name is either entity_extraction_agent or relation_extraction_agent).
2. Finish the conversation using: <goto>FINISH</goto> and include the final result.

Guidelines:
- Do not provide any extraction information yourself; rely solely on the agents.
- The very first <goto> tag in your response will be executed.
- Only include one agent call in your response.
- Providing no output will result in a NoneType Error.

This is the text to process: {text}
"""


def supervisor(state):
    # Stream the response and accumulate the tokens
    response = model.invoke(state["messages"])
        
    
    
    # Now extract the <goto> tag from the accumulated response
    goto_match = re.search(r'<goto>(.*?)</goto>', response.content)
    if goto_match:
        goto = goto_match.group(1)
    else:
        goto = "supervisor"
        
    print(f"\n\n-- START OF OUTPUT (SUPERVISOR) --\n\n{response.content}\n\n-- END OF OUTPUT (NEXT: {goto}) --\n\n")
    
    return state["messages"] + [response]

messages = supervisor({"messages": [{"role": "system", "content": system_prompt}], "text": text})
messages = supervisor({"messages": messages + [{"role": "assistant", "content": "Output of entity_extraction_agent: [Olaf Scholz, chancellor, Germany]"}], "text": text})
messages = supervisor({"messages": messages + [{"role": "assistant", "content": "Output of relation_extraction_agent: [[Olaf Scholz,is chancellor of, Germany], [Olaf Scholz, has job, chancellor]]"}], "text": text})



-- START OF OUTPUT (SUPERVISOR) --

<goto>entity_extraction_agent</goto>

-- END OF OUTPUT (NEXT: entity_extraction_agent) --




-- START OF OUTPUT (SUPERVISOR) --

<goto>relation_extraction_agent</goto>

-- END OF OUTPUT (NEXT: relation_extraction_agent) --




-- START OF OUTPUT (SUPERVISOR) --

<goto>FINISH</goto> The final result is: [[Olaf Scholz,is chancellor of, Germany], [Olaf Scholz, has job, chancellor]]

-- END OF OUTPUT (NEXT: FINISH) --


